# Carlos Tardón Rubio y Carlos Morán Alfonso

# Práctica 1.A Toma de contacto con AIMA

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se **representan** algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  

Cuando termines los ejercicios entrega este archivo en el campus. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [1]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [2]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [ ]:
p =Jarras()
p.initial

In [ ]:
p.actions(p.initial)

In [ ]:
p.result(p.initial,"llenar jarra de 4")

In [ ]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

In [ ]:
p.goal_test(p.initial)

### Problema de los misioneros

In [ ]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [ ]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [ ]:
misioneros.initial

In [ ]:
misioneros.actions(misioneros.initial)

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [3]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        
        
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        else:
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0

        
       ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [ ]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

In [ ]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [ ]:
p8.result(p8.initial,"Mover hueco arriba")

In [ ]:
p8.result(p8.initial,"Mover hueco abajo")

In [ ]:
p8.result(p8.initial,"Mover hueco derecha")

In [ ]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [6]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *

In [ ]:
## resolvemos el problema de las jarras con el método de búsqueda en anchura.  

In [ ]:
%%timeit
breadth_first_tree_search(Jarras()).solution()

In [ ]:
%%timeit
depth_first_graph_search(Jarras()).solution()

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8

In [ ]:
%%timeit
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura
breadth_first_tree_search(misioneros).solution()

In [ ]:
%%timeit
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura
depth_first_tree_search(misioneros).solution()

In [ ]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

In [ ]:
p8.goal

In [ ]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
# La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
# Busqueda en anchura es completo.. ¿no debería terminar? escribe al final del ejercicio tus conclusiones.

In [ ]:
%%timeit
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

In [ ]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [ ]:
%%timeit
breadth_first_tree_search(estado).solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']

In [ ]:
%%timeit
depth_first_graph_search(estado).solution()

In [ ]:
%%timeit
breadth_first_graph_search(estado).solution()

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones:


Datos recogidos:

Problema de las jarras:
La instancia del problema de las jarras tiene solución y, por lo tanto, tanto la búsqueda en anchura como la búsqueda en profundidad van a encontrar solución siempre (por su completitud). Es decir, la única cuestión es el tiempo. Usando la función timeit, estos son los valores de la media y la desviación típica de la ejecución de estos dos algoritmos (en todos los casos los datos se basan en 7 ejecuciones):
Búsqueda en anchura: Media (3,98ms). Desviación típica (174 μs)
Búsqueda en profundidad: Media (80,8 μs). Desviación típica (2,73 μs)
Cabe destacar también que Se nota bastante el control de repeticiones del graph_search, pues pasa de 67.1 µs a 2.29 ms si la busqueda es tree_search


Problema de los misioneros:
Como tenemos un criterio de validez para una instancia de este problema (que no haya más caníbales que misioneros), todos los casos en los que se pueda hacer búsqueda en anchura o en profundidad se va a encontrar solución (las instancias "imposibles" quedan invalidadas previamente). Con la función timeit hemos calculado la media de las ejecuciones y la desviación típica de las mismas (en base a 7 ejecuciones):
Búsqueda en anchura: Media (184ms). Desviación típica (15,7ms)


Puzle del 8:
Instancia (2,8,3,1,6,4,7,0,5): No termina dado que no existe solucón en este caso (el criterio en esta ocasión es si el número de inversiones es par o no). Como no existe solución, la búsqueda en anchura no termina (la completitud de la búsqueda en anchura nos indica que siempre encontrará solución si ésta existe).

Instancia (2,4,3,1,5,6,7,8,0): En este caso si tenemos solución. De nuevo en base a 7 ejecuciones, tenemos resultados de búsqueda en anchura (en árbol y en grafo) y búsqueda en profundidad (sólo en árbol):
Búsqueda en anchura (en árbol): Media (23,6ms). Desviación típica (2,56μs)
Búsqueda en anchura (en grafo): Media (2,28ms). Desviación típica (127μs)
Búsqueda en profundidad: Media (14min 7s). Desviación típica (20min 17s)
Con respecto a ésta última, hay que decir que encuentra solución pero no es óptima. Hay que destacar que el orden en el que se añadan las acciones en la función result influye en el resultado, pues con otros órdenes se obtienen soluciones distintas.


La conslusión que sacamos es que la búsqueda ciega es ineficiente en general pero la búsqueda en profundidad es aún más ineficiente en particular (en la última instancia del puzle del 8, vemos que cuantos más nodos haya que generar, más ineficiente va a ser esta búsqueda porque los visita todos sin ningún tipo de criterio)


### Ejercicio 3:  Definición de heurísticas

#### Para el problema de los misioneros define una heurística y comenta sus propiedades
#### Para el problema del puzle de 8 se pide definir al menos las siguientes funciones heurísticas:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node): maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [4]:
# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 


def pos(ind):
    return int(ind / 3), ind%3

def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    return len([1 for i,j in zip(goal,node.state) if i!=j])

def manhattan(node):
    state = node.state
    mhd = 0
    for i,e in enumerate(state):
        x,y = pos(i)
        correct_i = e-1 if (e>0) else 8  # assuming goal = (1,2,3,4,5,6,7,8,0)
        x1,y1 = pos(correct_i)
        mhd += abs(x1-x) + abs(y1-y)
    
    return mhd

def sqrt_manhattan(node):
    mhd = manhattan(node)
    
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

In [ ]:
puzle8 = Ocho_Puzzle((2,4,3,1,5,6,7,8,0))

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente **estado inicial** y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [ ]:
%%timeit
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzle).solution()

In [ ]:
%%timeit
astar_search(puzle,manhattan).solution()

In [ ]:
puzle.initial

In [ ]:
puzle.goal

In [ ]:
%%timeit
astar_search(puzle,linear).solution()

In [ ]:
%%timeit
astar_search(puzle,max_heuristic).solution()

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo.Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo
<br>

In [8]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [ ]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

In [ ]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

In [9]:

astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

NameError: name 'math' is not defined

In [ ]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.


La heurística lineal da un tiempo medio de 2.66 ms, mayor que los 2.16 ms que se tarda usando la heurística manhattan. Esto es esperable, pues la manhattan es más informada que la lineal. 

En cuanto a sqrt_manhattan, no ahorra ningún esfuerzo de búsqueda con respecto a la manhattan original (pues sqrt_manhattan(x) < sqrt_manhattan(y) <=> manhattan(x) < manhattan(y)), y además, supone un gasto extra en cada nodo, al tener que computar la raiz cuadrada de la manhattan, y por ello es que el coste medio es de 36.8 ms. 

Por último, max_heuristic es más informada que las anteriores(de hecho, equivale a la manhattan, al ser más informada que la lineal), pero debido a que tiene que hacer más calculos, max_heuristic tarda un poco más que la manhattan (2.39 ms)




## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



In [ ]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0        

In [ ]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [ ]:
p8p.initial

In [ ]:
p8p.goal

In [ ]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1,linear).solution()

In [ ]:
astar_search(p8, h2_ocho_puzzle).solution()

In [ ]:
astar_search(p8p, h2_ocho_puzzle).solution()

In [ ]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [ ]:
resuelve_ocho_puzzle(puzzle_1.initial,astar_search,sqrt_manhattan)

In [ ]:
E1 = (2,1,3,4,8,6,7,0,5)
E2 = (1,0,3,4,8,6,7,2,5)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E4,breadth_first_search)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E2,depth_first_graph_search)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E3,uniform_cost_search)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E4,best_first_graph_search,linear)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E4,best_first_graph_search,manhattan)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E4,astar_search,linear)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E1,astar_search,manhattan)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E2,best_first_graph_search,manhattan)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E3,best_first_graph_search,manhattan)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E4,best_first_graph_search,manhattan)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E1,astar_search,manhattan)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E2,astar_search,manhattan)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E3,astar_search,manhattan)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E1,astar_search,manhattan)

### Ejercicio 5:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

E1 = (2,1,3,4,8,6,7,0,5)
E2 = (1,0,3,4,8,6,7,2,5)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)

          E1              E2              E3              E4
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
     | 2 | 1 | 3 |   | 1 | 0 | 3 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 4 | 8 | 6 |   | 4 | 8 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 7 | H | 5 |   | 7 | 2 | 5 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la
 siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo, y
 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  
 
 

-----------------------------------------------------------------------------------------
                                       E1           E2           E3         E4
    Anchura                            L=17         L=11         L=20        L=3
                                       T=26s        T=51,8ms     T=2min 45s  T=2,68ms  
                                       NA=12936     NA=853       NA=52489    NA=19 
   
    Profundidad                        L=60357      L=57847      L=60022     L=3  
                                       T=N/R        T=N/R        T=N/R       T=534 µs 
                                       NA=73823     NA=68752     NA=73091    NA=4 
                                       
    Coste Uniforme                     L=17         L=11         L=20        L=3
                                       T=1min 5s    T=205ms      T=N/R       T=2,78ms  
                                       NA=14093     NA=871       NA=48428    NA=14 
                                       
    Primero el mejor (linear)          L=39         L=33         L=76        L=3
                                       T=10,8ms     T=3.48ms     T=250ms     T=4,1ms  
                                       NA=174       NA=79        NA=749      NA=5
                                                                            
    Primero el mejor (manhattan)       L= 67        L=15         L=74        L= 3
                                       T=113 ms     T=1.17 ms    T=109 ms    T= 465 µs 
                                       NA= 652      NA=27        NA=592      NA=15
                                                                             
    A* (linear)                        L=17         L=11         L=20        L=3 
                                       T=205ms      T=4,32ms     T=7,67s     T=3,23  
                                       NA=874       NA=78        NA=3378     NA=5
                                                                             
    A* (manhattan)                     L=17         L=11         L=20        L=3 
                                       T=63,5ms     T= 1.72 ms   T=306 ms    T=492 µs  
                                       NA=362       NA=40        NA= 1154    NA=5
                                      
                                      
                                          
     A* (h3)                           L=17         L=11          L=20        L=3 
                                       T=82.7 ms    T= 2.1 ms     T=487 ms    T=925 µs  
                                       NA=332       NA=19         NA= 861     NA=5

 -----------------------------------------------------------------------------------------

Observaciones:

Notación: N/R (no razonable)

Como ya discutimos en el ejercicio 3, la búsqueda en profundidad es el algorimo menos eficiente a la hora de resolver el 8-puzle. Aquí vemos que ni es eficiente (siendo la que más tarda con diferencia en los 4 casos y en algunos casos no llega a acabar) ni es óptima (la solución proporcionada en E1 no es para nada óptima).


Algo parecido (aunque no a tan gran escala y en este caso sí que es óptimo) podemos observar con la búsqueda por coste uniforme. En este caso solo una instancia no ha podido ser completada en tiempo razonable (E3) pero en todos los casos siemmpre es éste el algoritmo que más tiempo ha invertido (sin contar profundidad)


En cuanto a optimalidad, vemos que los algoritmos de búsqueda ciega comprobados (búsqueda en anchura y coste uniforme)  son óptimas aunque exploran una cantidad de nodos mucho mayor que el resto de algoritmos. Además, por ser las heurísticas admisibles y consistentes, A* encuentra la solución óptima, en un tiempo mucho menor que anchura o coste uniforme. Mientras tanto, los algoritmos voraces con heurísticas (el mejor primero) son los mejores en cuanto a tiempo de ejecución pero solo han conseguido encontrar la solución óptima en 1 de los 4 casos (E4) aunque hay que destacar que primero el mejor con linear encuentra una solución explorando tan solo 173 nodos en E1.

Como no existen caminos de coste infinito, y los costes son positivos (1 cada acción),la búsqueda uniforme es completa y óptima(por ejemplo, encuentra el camino de longitud 17 en el problema E1).




### Ejercicio 6:  En el ejercicio anterior hemos hecho pruebas con distintos estados iniciales. Explica cómo afecta a la resolución del problema si lo que cambiamos es el estado objetivo.
####  En vez de  goal=(1, 2, 3, 4, 5, 6, 7, 8, 0) queremos usar goal2=(1, 2, 3, 4, 0, 5, 6, 7, 8)
####  No es necesario repetir los experimentos pero sí justificar convenientemente la respuesta dada.




Si cambiamos el estado objetivo, lo primero que habría que hacer es modificar las heurísticas, pues las actuales solo funcionan para el estado final  goal=(1, 2, 3, 4, 5, 6, 7, 8, 0) 

Dado que en ningún caso cambia el número de inversiones (para las inversiones no se tiene en cuenta el hueco), la única diferencia será en el tiempo de ejecución. Ninguna solución previamente imposible será posible y viceversa.

 Como las acciones del 8-puzzle tienen inversa, resolver el puzzle hacia delante es equivalente a resolverlo hacia atrás(empezando por goal o goal2). 

El primer problema que puede surgir al cambiar de estado objetivo es que pase a no ser alcanzable. Sin embargo, en la siguiente celda se comprueba que goal es alcanzable desde goal2, así que todas las búsquedas completas que tengan solución con goal también tienen solución con goal2. Además, como profundidad, coste uniforme y a* son óptimas, encontrarán solucion eficiente en este nuevo problema, a* en menor tiempo. De profundidad y primero el mejor no podemos asegurar nada.



In [ ]:
state = (1, 2, 3, 4, 0, 5, 6, 7, 8)
resuelve_ocho_puzzle(state,astar_search,manhattan)

### Ejercicio 7: (opcional) Definir nuevas heurísticas (más informadas) y completa una nueva fila de la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

In [ ]:
# h cuantifica todas las inversiones, es decir, los pares de elementos(sin contar el 0) del estado actual tal que
# su orden está invertido con respecto a la solucion final goal=(1, 2, 3, 4, 5, 6, 7, 8, 0) 
# por ejemplo h((1, 2, 3, 4, 5, 6, 7, 0, 8)) = 1 porque el orden del 8 y del 0 estan cambiados
# esta nueva heuristica no es admisible, pues h((1, 2, 3, 4, 5, 0, 7, 8, 6)) > 1 = coste. Esto hace que la solución
# encontrada pueda no ser óptima. En los casos de ejemplo, encuentra la óptima
def h(nodo):
    state = nodo.state
    count = 0
    for i in state:
        for j in state[(state.index(i)+1):]:
            if j > 0:
                if j < i :
                    count += 1
    return count

In [ ]:
# como h no es más informada que manhattan, hacemos el máximo para que h3 sea más informada
def h3(nodo):
    return max(h(nodo),manhattan(nodo))

    A* (h3)                     L=17         L=11          L=20        L=3 
                                T=82.7 ms    T= 2.1 ms    T=487 ms    T=925 µs  
                                NA=332       NA=19         NA= 861     NA=5
                                
#### Por tanto, h3 recorre menos nodos, aunque los tiempos den mayores(puede ser por el cálculo extra que supone calcular h en cada nodo)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E2,astar_search,h3)

In [ ]:
%%timeit
resuelve_ocho_puzzle(E3,astar_search,manhattan)

### Ejercicio 8. Puzle de 15.
En 1878, Sam Loyd daba un premio de 1.000 dolares a quien fuera capaz de resolver su famoso 15 Puzzle (n=16). Se trataba de un puzzle deslizante de 16 piezas que *no tenía solución* debido que se necesitaba un número impar de movimientos y, como hemos visto, sólo un número de movimientos par tiene solución. Muchas personas trataron de resolverlo. 
El número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. 
En el caso del siguiente estado inicial que sí tiene solución:  (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


In [ ]:
class Quince_Puzzle(Problem):

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2,3):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (3,7,11,15):
            accs.append("Mover hueco derecha")
        if pos_hueco not in (12,13,14,15):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,4,8,12):
            accs.append("Mover hueco izquierda")
        
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        
        
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-4]
            l[pos_hueco-4] = 0
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+4]
            l[pos_hueco+4] = 0
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        else:
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0

        
       ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        
        
        return tuple(l)
    
    ## Obtiene la fila donde está el elemento pedido (usado para ver si se puede resolver o no)
    def row(self,index):
        return int(index/4)


    ## Un estado inicial del puzle del 15 se puede resolver si se da 1 de las 2 siguientes condiciones:
    # 1. El número de inversiones es impar y el hueco está situado (en el estado inicial) en una fila par contando desde abajo
    # 2. El número de inversiones es par y el hueco está situado (en el estado inicial) en una fila impar contando desde abajo

    def check_solvability_15(self, state):

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return (inversion % 2 == 0 and self.row(state.index(0)) % 2 != 0) or (inversion % 2 != 0 and self.row(state.index(0) % 2 == 0))      
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

In [ ]:
def resuelve_quince_puzzle(estado_inicial, algoritmo, h=None):

    p15 = Quince_Puzzle(estado_inicial)
    p15p=Problema_con_Analizados(p15)
    if p15.check_solvability_15(estado_inicial):
        if h: 
            sol= algoritmo(p15p,h).solution()
        else: 
            sol= algoritmo(p15p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p15p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [ ]:
def linear_15(node):
    goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
    return len([1 for i,j in zip(goal,node.state) if i!=j])

In [ ]:
puzle_15 = Quince_Puzzle((1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0))
resuelve_quince_puzzle(puzle_15.initial,astar_search,linear_15)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear_15
Longitud de la solución: 4. Nodos analizados: 5

In [ ]:
resuelve_quince_puzzle(puzle_15.initial,breadth_first_search)

In [ ]:
puzle_8 = Ocho_Puzzle((1,2,3,4,8,5,7,6,0))
resuelve_ocho_puzzle(puzle_8.initial,astar_search,linear)


Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 4. Nodos analizados: 6

##### Vemos que para soluciones parecidas de corta longitud, los nodos analizados en ocho puzzle y en quince puzzle son parecidos. Sin embargo, en cuanto aumenta la complejidad, quince puzzle es más caro computacionalmente